In [ ]:
import tensorflow as tf

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
checkpoint = "google-t5/t5-small"

## Helsinki-NLP/opus-mt-en-hi model

source: https://huggingface.co/Helsinki-NLP/opus-mt-en-hi



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [ ]:
# from datasets import load_dataset

raw_datasets = load_dataset("opus_books", "en-fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [ ]:
# Split the original dataset into training and testing sets
subset_dataset = raw_datasets["train"].train_test_split(test_size=0.1)

In [ ]:
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 114376
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 12709
    })
})

In [ ]:
subset_dataset['train']['translation'][:2]

[{'en': '"A year ago I was myself intensely miserable, because I thought I had made a mistake in entering the ministry: its uniform duties wearied me to death.',
  'fr': "«Il y a un an, moi aussi j'étais bien malheureux, parce que je croyais m'être trompé en entrant dans les ordres; l'uniforme du prêtre et ses devoirs me pesaient;"},
 {'en': '"Pardon me, father," I said, "but I shall not come."',
  'fr': '--Pardonnez-moi, mon père, dis-je alors, mais je ne partirai pas.'}]

#Preprocessing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Le grand Meaulnes"]))

{'input_ids': [[312, 1907, 1212, 9, 83, 1496, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
num_classes = tokenizer.vocab_size
num_classes

32100

In [ ]:
source_lang = "en"
target_lang = "fr"
max_input_length=128
max_target_length =128


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True,padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
subset_dataset['test'][:2]

{'id': ['11205', '119658'],
 'translation': [{'en': '"He is not at home."', 'fr': "-- Il n'est pas ici."},
  {'en': 'They had both embraced again, and as they were pressing each other in a long clasp they heard steps, which made them get up.',
   'fr': "Tous deux s'étaient repris; et, comme ils se serraient d'une longue étreinte, des pas les firent se mettre debout."}]}

In [ ]:
tokenized_book = subset_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/114376 [00:00<?, ? examples/s]

Map:   0%|          | 0/12709 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
batch_size = 16

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_book['test'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 128), dtype=tf.int64, name=None))>

In [ ]:
# validation_dataset = model.prepare_tf_dataset(
#     tokenized_val,
#     batch_size=batch_size,
#     shuffle=False,
#     collate_fn=data_collator,
# )

In [ ]:
train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 128), dtype=tf.int64, name=None))>

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, epochs=100)

Epoch 1/100
794/794 [==============================] - 267s 311ms/step - loss: 1.8101
Epoch 2/100
794/794 [==============================] - 250s 315ms/step - loss: 0.8514
Epoch 3/100
794/794 [==============================] - 250s 315ms/step - loss: 0.7460
Epoch 4/100
794/794 [==============================] - 250s 314ms/step - loss: 0.6996
Epoch 5/100
794/794 [==============================] - 250s 315ms/step - loss: 0.6742
Epoch 6/100
794/794 [==============================] - 250s 314ms/step - loss: 0.6549
Epoch 7/100
794/794 [==============================] - 250s 314ms/step - loss: 0.6397
Epoch 8/100
794/794 [==============================] - 250s 314ms/step - loss: 0.6261
Epoch 9/100
118/794 [===>..........................] - ETA: 3:33 - loss: 0.6242

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("tf_model/")

# Model Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
input_text  = "hi machi how are you"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[   0    3   18    3 8276   23    6 1670    3 6738   18 3249   58    1]], shape=(1, 14), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0],skip_special_tokens=True))

- machi, comment êtes-vous?


In [ ]:
out[0]

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([0, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3,
       2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3], dtype=int32)>